# Part 2 of the Investigation into "their kingdom" of 4Q246 II.2

<img src="images/their_kingdom.png"></img>

In [1]:
from datetime import datetime
print(f'Notebook last updated on {datetime.now().__str__()}')

Notebook last updated on 2017-05-18 15:51:13.178548


## Introduction 
In the previous notebook, [their_kingdom_1.ipynb](their_kingdom_1.ipynb), a query aimed to address whether the 3MP from יקרונה ("**they** will name him") in col II line 1 of 4Q246 was likely to be the same 3MP in the subsequent clause, כן מלכותהן תהוה ("thus will **their** kingdom be." see the image above). To do so, the query surveyed cases in the Hebrew Bible wherein the PGN markers were the same but had different referents.

Out of 405 matches<sup>1</sup> in Biblical Hebrew, 5 instances contained a verb with an impersonal subject which led to a disconnect between the two PGN markers. 2 of those examples were due to a passive verb. Further investigation into the phenomenon of impersonal subjects in Aramaic led to the recognition that the 3MP is commonly used in such a way in Aramaic, often as a parallel to passive verbs. Those same circumstances exist in 4Q246 II.1, and that would suggest the possibility that the PGN of the verb and following pronominal suffix are not actually referring to the same referent.

In such a situation, it is necessary to account for a new antecedent for the "their" of מלכותהן. One possibility is provided by J.A. Fitzmeyer, who interprets the extant phrase, מלך אתור ומצרין ("king of Assyria and Egypt") as a reference to *two* kings. His reconstruction of the phrase is as follows:

> ולקצת יומיא יאבדון] מלך אתור [ומ]צרין] <br>
> [And at the end of the days] the king of Assyria [and Eg]ypt [(they) will perish].<sup>2</sup>

Fitzmeyer's suggestion does not receive a defense or explanation. On the other hand, it offers a interpretation of the phrase מלך אתור ומצרין ("king of Assyria and Egypt") which could provide an apt antecedent for the pronominal suffix on מלכותהן ("their kingdom"). If such is the case, an additional query will search for the role of semantics in linking PGN indicators.

If Fitzmeyer's suggestion is not viable, though, it suggests some added difficulty for the positive interpretation. The lack of another extant plural subject might boost the likelihood that the antecedent of the plural 3MP of מלכותהן is the same 3MP that serves (יעבדון) the "son of God" (I.8), since that is the nearest alternative 3MP in the text.
<br>
<br>
<hr width="500" align="left">
<sup>1</sup>&nbsp;&nbsp;Clauses matched were those with a third person verb followed by a clause with a third person, identical pronominal suffix, without interruption by another PNG marker

<sup>2</sup>&nbsp;&nbsp;Joseph A. Fitzmeyer, *The Dead Sea Scrolls and Christian Origins* (Grand Rapids: 2000), 44.

<hr>

## Research Question

Is the following construction attested in the Hebrew Bible?

* plural_verb + construct_noun_singular + absolute_noun_singular + coordinate_waw + absolute_noun_singular
* i.e. "they did X" + (subj) "something of" + "person 1" + and + "person 2" 
    * or: "king of Assyria and Egypt [they] will perish."
    
    
## Query Strategy

We query a list of clauses and look for two groups. Group 1 is the most stringent search parameters. Group 2 is less strict.

**Group 1** - Clauses that have
1. a plural predicate
2. a singular subject in construct state
    * followed by coordinated nouns
    
**Group 2** - Clauses that have
1. a plural predicate
2. a singular subject

We search the whole HB, Hebrew and Aramaic together.

Results are printed or exported to a spreadsheet, depending on the number of results.

In [2]:
# begin code

# import tools
import collections, csv

# import text fabric
from tf.fabric import Fabric

# instantiate TF processor
TF = Fabric(modules='hebrew/etcbc4c')

# load TF features
api = TF.load('''
              book chapter verse
              nu function pdp rela lex
              mother kind typ st
              ''')

# prevent having to prefix all functions with .api
api.makeAvailableIn(globals())

This is Text-Fabric 2.3.7
Api reference : https://github.com/ETCBC/text-fabric/wiki/Api
Tutorial      : https://github.com/ETCBC/text-fabric/blob/master/docs/tutorial.ipynb
Data sources  : https://github.com/ETCBC/text-fabric-data
Data docs     : https://etcbc.github.io/text-fabric-data
Shebanq docs  : https://shebanq.ancient-data.org/text
Slack team    : https://shebanq.slack.com/signup
Questions? Ask shebanq@ancient-data.org for an invite to Slack
109 features found and 0 ignored
  0.00s loading features ...
   |     0.01s B book                 from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B chapter              from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B verse                from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.11s B nu                   from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.07s B function             from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.11s

## Get Nomens Regens/Rectums Function

First we need a function which can isolate the individual, coordinated elements within a nomens rectum (i.e. the genitive(s) in a construct relation). This allows us to check and see whether one or many elements are coordinated with a nomen regens (the head word in the genitive construction).  

In ETCBC4c, [only the the nomens rectum relationship is indicated on subphrase objects](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/rela). In this function, we find the regens and then use it as a reference point to find the corresponding rectums.

In [27]:
def get_regens_rectums(ca):
    '''
    Return a dict of the nomen regens and their respective rectums
    from within a clause atom.
    The dict contains the regens (word node) as the key,
    A list of corresponding nomen rectum subphrases is the value.
    Require a clause atom node.
    '''
    
    # map nomen regens to their nom. rectums here
    reg_to_rec = {}
    
    # gather all nomen regens in the clause atom
    nomen_regens = [word for word in L.d(ca, otype='word')

                        # children have nomens rectum relationship
                        if 'rec' in {F.rela.v(sp) for sp in E.mother.t(word)}
                    ]

    # gather the nom. rectums
    for regens in nomen_regens:
        
        # get the nomen rectum (including the individual parts)
        # there will only be 1 match for a given nom. regem
        rectum = [rec for rec in E.mother.t(regens) 
                      if F.rela.v(rec) == 'rec'][0]
        
        # get the first word in the nomen rectum subphrase
        first_rectum_word = L.d(rectum, otype='word')[0]
        
        # get the first rectum's subphrase, but the one that is not 'rec'
        # this is to isolate the parallel subphrases
        rectum_in_series = [sp for sp in L.u(first_rectum_word, otype='subphrase')
                                if F.rela.v(sp) == 'NA']
        
        
        # the nomen regens is the only element
        # save and map in the dictionary
        if not rectum_in_series:
            reg_to_rec[regens] = [rectum]
        
        
        # the nomen regens may be the first element in a series
        # gather the other elements in the series and save them
        else:
            # collect rectums here
            all_rectums = []
            
            # for each rectum subphrase, look for a daughter subphrase that
            # is in a parallel relation that itself 
            # is contained within the nomens 'rectum'
            for rectum in rectum_in_series:

                # get the children of the first rectum that are 'parallel'
                # filter out any results that are not within the 'rectum' subphrase
                add_rectums = [sp for sp in E.mother.t(rectum)
                               
                                   # subphrase must be in a parallel relation
                                   if F.rela.v(sp) == 'par'
                               
                                   # first word in subphrase must be contained
                                   # within the rectum subphrase
                                   and 'rec' in set(F.rela.v(sp) for sp in 
                                                     L.u(
                                                         L.d(sp, otype='word')[0], # 1st wrd
                                                         otype='subphrase'
                                                        ) 
                                                    )
                              ]
                
                # add the additional rectums to the complete list
                all_rectums.extend(add_rectums)
                
            # add in the first rectum
            all_rectums.insert(0, rectum_in_series[0])
                
            # map the rectums to regens
            reg_to_rec[regens] = all_rectums 

    return reg_to_rec

## Validate PGN Functions

Before subject/verb agreement can be compared, we have to isolate the carriers of PGN data (usually a noun) from within the subject phrase(s). This problem requires several work-arounds and is as yet not fully solved, since there are multiple edge cases. These two algorithms are well-deserving of their own notebooks and further exploration. For the purposes here, however, they have been tested with good results and are more than sufficient for the goal of this notebook.

In [23]:
def is_preposition_subj(word):
    '''
    Return boolean on whether a word is a preposition subject,
    necessary for cases in which the subject is marked in
    a prepositional phrase, such as in passive clauses.
    Require a word node.
    
    *Caution*
    Does not capture cases such as Gen 21:5 (ca# 516487)
    '''
    # get word phrase
    w_phrase = L.u(word, otype='phrase')[0]
    
    # return false if wordnode not in a subject phrase
    if F.function.v(w_phrase) != 'Subj':
        return False
    
    # get all phrase atoms in the phrase    
    # exclude negations and conjunctions
    phrase_atoms = [phrs_at for phrs_at in L.d(phrase, otype='phrase_atom')
                        if F.typ.v(phrs_at) not in {'NegP','CP'}
                   ]
    
    # check whether the only phrase atom in the phrase is a prep. phrase
    if len(phrase_atoms) == 1 and F.typ.v(phrase_atoms[0]) == 'PP':
        
        # is a prepositional subject
        return True

    else:
        # is not a prep subj
        return False
    
    
def validate_subject(word):
    '''
    Return boolean on whether a word is a subject or not,
    i.e. a word without any modifiers that functions as subj.
    Require word node.
    
    Based on a supplied wordnode get phrase, phrase atom, and subphrase,
    features and compare them against a group of sets.
    Define those sets first. Then make the comparison.
    
    *Caution* 
    This function works reasonably well,
    but there are a number of edge cases that it does not catch.
    Fine-tuning this function would make a nice notebook in itself.
    See Gen 20:5 for a good edge case example, in which both היא pronouns
    are registered as subjects, but only one should be.
    '''
    
    # keep words with these part of speech tags
    keep_pdp = {'subs', # noun
                'nmpr', # proper noun
                'prps', # personal pronoun
                'prde', # demonstrative pronoun
                'prin'} # interrogative pronoun
                   
    # keep words in phrase_atoms with these type features
    keep_pa_typ = {'NP',   # noun phrase
                   'PrNP', # proper noun phrase
                   'PPrP', # personal pronoun phr.
                   'DPrP', # demonstrative pron. phr.
                   'IPrP'} # interrogative pron. phr.
    
    # exclude words in phrase_atoms with these relation features
    omit_pa_rela = {'Appo', # apposition
                    'Spec'} # specification
    
    # exclude words in subphrases with these relation features
    omit_sp_rela = {'rec', # nomens rectum
                    'adj', # adjectival 
                    'atr', # attributive
                    'mod', # modifier
                    'dem'} # demontrative
                        
    # get word's phrase, phrase atom, and subphrase, and subphrase relations
    w_phrase = L.u(word, otype='phrase')[0] # word's phrase node
    w_phrase_atom = L.u(word, otype='phrase_atom')[0] # word's phrase atom
    w_subphrases = L.u(word, otype='subphrase') # word's subphrase
    w_subphrs_relas = set(F.rela.v(sp) for sp in w_subphrases) # subphrs rela
    
    # compare word/phrase features against the sets
    if all([
            # word in subj phrase
            F.function.v(w_phrase) == 'Subj', 
            
            # phrase dependent part of speech is valid
            F.pdp.v(word) in keep_pdp,
            
            # either the word is in good phrase atom type 
            # or is part of a prepositional subj, e.g. with passives
            F.typ.v(w_phrase_atom) in keep_pa_typ\
                or is_prepositional_subj(word),
            
            # phrase_atom relation is valid
            F.rela.v(w_phrase_atom) not in omit_pa_rela,
            
            # subphrase relation is valid
            not w_subphrs_relas & omit_sp_rela,
           ]):
        
        # word is a subject
        return True
    
    else:
        # word is not subject
        return False

## Full Function

Now we write a full function to identify clause atoms that meet the requirements outlined above.

In [153]:
def find_nomen_rectum_subjs(ca):
    '''
    Look for plural subjects in a nomens rectum.
    First, check for subject/verb disagreement in which:
        * Verb is plural.
        * Subject is singular.
    Second, check for coordinated nomen rectums in which
        the subject is the mother.
    '''                      
    
    # get the verb; assign here
    verb = None 
    
    # get predicate phrase and pull its verb
    for phrase in L.d(ca, otype='phrase'):
        
        # skip non-predicate phrases
        if F.function.v(phrase) not in {'Pred','PreO'}:
            continue
    
        # find verb's word node
        for word in L.d(phrase, otype='word'):
            
            # assign verb
            verb = word if F.pdp.v(wrd) == 'verb' else verb
    
    
    # get the subject with validate_subject
    subj = [wrd for wrd in L.d(ca, otype='word')
               if validate_subject(wrd)]
    
    # skip if no verb/no subj
    if not verb or not subj:
        continue

    # get number for verb/subj(s)
    verb_nu = F.nu.v(verb)
    subj_nu = F.nu.v(subj[0]) if len(subj) == 1 else 'pl'

    # return False if numbers are different
    if any([verb_nu == 'pl' and subj_nu == 'sg',
            verb_nu == 'sg' and subj_nu == 'pl'
           ]):

    # numbers are same
    else:

SyntaxError: 'continue' not properly in loop (<ipython-input-153-bf8e13840328>, line 47)

See especially Gen 14:10! That is the kind of result we will be seeking.